<a href="https://colab.research.google.com/github/pravincoder/binance_trade_analysis/blob/main/raw_csv_to_trade_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
url = "https://github.com/pravincoder/binance_trade_analysis/raw/refs/heads/main/TRADES_CopyTr_90D_ROI.csv"
df = pd.read_csv(url)



In [18]:
# Lets focus on the trade_hist data in each row in each account and a single trade param
df.Trade_History.head(1).to_list()[0][0]

{'time': 1718899656000,
 'symbol': 'SOLUSDT',
 'side': 'BUY',
 'price': 132.537,
 'fee': -0.9940275,
 'feeAsset': 'USDT',
 'quantity': 1988.055,
 'quantityAsset': 'USDT',
 'realizedProfit': 0.0,
 'realizedProfitAsset': 'USDT',
 'baseAsset': 'SOL',
 'qty': 15.0,
 'positionSide': 'LONG',
 'activeBuy': True}

In [7]:
import ast
print(df.info())
print(df.head())

# Check if Trade_History column contains valid data
def parse_trade_history(history):
    try:
        return ast.literal_eval(history) if isinstance(history, str) else []
    except Exception as e:
        return []

# Apply parsing function
df["Trade_History"] = df["Trade_History"].apply(parse_trade_history)

# Display parsed data
print("Sample Trade History:")
print(df["Trade_History"].head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB
None
              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...
Sample Trade History:
0    [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1    [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2    [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3    [{'time': 1718678214000, 'symbol'

In [8]:
# Extract trade details into separate rows
trade_data = []
for idx, row in df.iterrows():
    port_id = row["Port_IDs"]
    trade_history = row["Trade_History"]

    for trade in trade_history:
        trade_data.append({
            "Port_IDs": port_id,
            "time": trade.get("time", None),
            "symbol": trade.get("symbol", None),
            "price": trade.get("price", None),
            "quantity": trade.get("quantity", None),
            "trade_type": trade.get("side", None),  # Assuming 'side' is 'BUY' or 'SELL'
            "Profit/Loss": trade.get("realizedProfit", None)
        })

# Convert extracted data into a DataFrame
df_trades = pd.DataFrame(trade_data)

# Convert timestamp to readable date
df_trades["time"] = pd.to_datetime(df_trades["time"], unit="ms")

# Display transformed data
print("Processed Trade Data:")
print(df_trades.head(20))



Processed Trade Data:
               Port_IDs                time    symbol      price    quantity  \
0   3925368433214965504 2024-06-20 16:07:36   SOLUSDT  132.53700  1988.05500   
1   3925368433214965504 2024-06-20 16:06:58  DOGEUSDT    0.12182  1398.98088   
2   3925368433214965504 2024-06-20 16:06:58  DOGEUSDT    0.12182   197.47022   
3   3925368433214965504 2024-06-20 16:06:56  DOGEUSDT    0.12182    16.56752   
4   3925368433214965504 2024-06-20 16:06:56  DOGEUSDT    0.12182    92.21774   
5   3925368433214965504 2024-06-20 16:06:56  DOGEUSDT    0.12182   251.43648   
6   3925368433214965504 2024-06-20 16:06:56  DOGEUSDT    0.12182    43.61156   
7   3925368433214965504 2024-06-20 16:06:47   SOLUSDT  132.58500   795.51000   
8   3925368433214965504 2024-06-20 16:06:47   SOLUSDT  132.58500  1193.26500   
9   3925368433214965504 2024-06-20 13:21:24   FILUSDT    4.45400   697.05100   
10  3925368433214965504 2024-06-20 13:21:24   FILUSDT    4.45400     6.23560   
11  39253684332149

In [9]:
# Save to CSV for further analysis
#df_trades.to_csv("trades.csv", index=False)

In [16]:


# Handle missing values
df_trades.fillna(0, inplace=True)

# Calculate financial metrics
def calculate_metrics(group):
    total_positions = len(group)
    win_positions = (group["Profit/Loss"] > 0).sum()
    total_profit = group["Profit/Loss"].sum()
    total_investment = group["quantity"].sum()
    win_rate = win_positions / total_positions if total_positions > 0 else 0
    roi = (total_profit / total_investment) if total_investment > 0 else 0

    # Sharpe Ratio Calculation
    if len(group) > 1:
        returns = group["Profit/Loss"].pct_change().dropna()
        sharpe_ratio = returns.mean() / returns.std() if returns.std() > 0 else 0
    else:
        sharpe_ratio = 0

    # Maximum Drawdown (MDD) Calculation
    cumulative_profit = group["Profit/Loss"].cumsum()
    running_max = cumulative_profit.cummax()
    drawdown = cumulative_profit - running_max
    max_drawdown = drawdown.min()

    return pd.Series([total_positions, win_positions, win_rate, total_profit, roi, sharpe_ratio, max_drawdown],
                     index=["Total_Positions", "Win_Positions", "Win_Rate", "PnL", "ROI", "Sharpe_Ratio", "MDD"])

# Group by Port_IDs and calculate metrics
df_metrics = df_trades.groupby("Port_IDs").apply(calculate_metrics).reset_index()

# Ranking Algorithm
# Score = (ROI * 0.4) + (PnL * 0.3) + (Sharpe Ratio * 0.2) + (Win Rate * 0.1)
df_metrics["Score"] = (df_metrics["ROI"] * 0.4) + (df_metrics["PnL"] * 0.3) + \
                        (df_metrics["Sharpe_Ratio"] * 0.2) + (df_metrics["Win_Rate"] * 0.1)

df_ranked = df_metrics.sort_values(by="Score", ascending=False).head(20)

# Save results
df_metrics.to_csv("calculated_metrics.csv", index=False)
df_ranked.to_csv("top_20_accounts.csv", index=False)

# Generate a basic report
report = f"""Binance Trade Analysis Report
===================================

Total Accounts Analyzed: {len(df_metrics)}
Top 20 Accounts saved in 'top_20_accounts.csv'

Metrics Calculated:
- ROI (Return on Investment)
- PnL (Profit and Loss)
- Sharpe Ratio (Risk-adjusted return)
- Maximum Drawdown (MDD)
- Win Rate
- Win Positions
- Total Positions

Ranking based on weighted score:
- ROI: 40%
- PnL: 30%
- Sharpe Ratio: 20%
- Win Rate: 10%


Top 10 Accounts based on Score:
{df_ranked[["Port_IDs","ROI","PnL", "Score"]].head(10)}
"""

# print analysis report
print(report)


Binance Trade Analysis Report

Total Accounts Analyzed: 149
Top 20 Accounts saved in 'top_20_accounts.csv'

Metrics Calculated:
- ROI (Return on Investment)
- PnL (Profit and Loss)
- Sharpe Ratio (Risk-adjusted return)
- Maximum Drawdown (MDD)
- Win Rate
- Win Positions
- Total Positions

Ranking based on weighted score:
- ROI: 40%
- PnL: 30%
- Sharpe Ratio: 20%
- Win Rate: 10%


Top 10 Accounts based on Score:
                Port_IDs       ROI           PnL         Score  Sharpe_Ratio  \
96   4020204877254599680  0.003116  71998.855953  21599.690974           0.0   
75   3999240873283311617  0.005437  42574.473679  12772.396600           0.0   
98   4021669203289716224  0.004044  26427.331592   7928.232486           0.0   
50   3960874214179953664  0.002333  19567.471286   5870.293887           0.0   
17   3907081197088384000  0.010992  18015.997370   5404.865005           0.0   
107  4028701921959171840  0.004065  17601.401398   5280.476339           0.0   
48   3956076827719377409 

<ipython-input-16-b8c4a7563de5>:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_metrics = df_trades.groupby("Port_IDs").apply(calculate_metrics).reset_index()
